In [1]:
import os

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [2]:
from crewai import Agent
from crewai_tools import FileReadTool, DirectoryReadTool


class Agents:
    def markdownPathSearcher(self):
        return Agent(
            role="pathSearcher",
            goal="Finds the markdown files inside {file_path} path",
            backstory="You are fluent in Korean, and you are very good at finding markdown files.",
            allow_delegation=False,
            verbose=True,
            tools=[
                DirectoryReadTool(),
            ],
        )

    def imgPathSearcher(self):
        return Agent(
            role="pathSearcher",
            goal="Finds the img files inside {img_path} path",
            backstory="You are fluent in Korean, and you are very good at finding image files.",
            allow_delegation=False,
            verbose=True,
            tools=[
                DirectoryReadTool(),
            ],
        )

    def fileSelector(self):
        return Agent(
            role="fileSelector",
            goal="Search One file that fits {question}, and Make one file that should be collected the paths of all files associated with the file.",
            backstory="You are a file search expert and fluent in Korean. You have a great ability to read and analyze the details of the file.",
            allow_delegation=False,
            verbose=True,
            tools=[
                FileReadTool(),
            ],
        )

    def Imgextracter(self):
        return Agent(
            role="Imgextracter",
            goal="Search One file that fits {question}, and Make one file that should be collected the paths of all files associated with the file.",
        )

    def researcher(self):
        return Agent(
            role="Researcher",
            goal="Finds and summarizes the markdown files that can solve the {question}",
            backstory="You are fluent in Korean, and you are very good at finding and summarizing files. Summarized information is essential for problem solving",
            verbose=True,
            tools=[
                FileReadTool(),
            ],
            max_iter=10,
        )

In [3]:
from crewai import Task


class Tasks:
    def markdownPathSearch(self, agent):
        return Task(
            description="Finds ALL the markdown files and inside {file_path} path",
            expected_output="Your final answer MUST markdown file path",
            agent=agent,
            output_file="MarkdownPath.md",
        )

    def imgPathSearch(self, agent):
        return Task(
            description="Finds ALL the image files and inside {img_path} path",
            expected_output="Your final answer MUST image path",
            agent=agent,
            output_file="ImgPath.md",
        )

    def fileSelct(slef, agent, context):
        return Task(
            description="""
            Based on the markdownPathSearch and imgPathSearch, Search Only one file that can solve {question}. There are other documents linked by the symbol '[[...]]' and '![[...]]' in that file NOT '[...]'. 
            '[[...]]' symbol means a markdown file and '![[...]]' means an image file.
            
            Find all of the '[[...]]' and '![[...]]' and print out the ONLY file path associated with the word in it in markdownPathSearch or imgPathSearch. 
            All file paths should EXIST in that markdownPathSearch Output or imgPathSearch Output. 
            If the relevant document/image does not exist, JUST Say 'NOTHING'.",
            """,
            expected_output="""
            Your final answer MUST include the path of the first file and the path of other files within that file.
            It doesn't include ANYTHING other than file paths. 
            Example Answer 1
            Main file
            ./Algorithm/Algorithm Content/Tree/MST(Minimum Spanning Tree).md
            Related Files
            ./Algorithm/Algorithm Content/Graph Theory/DFS(Depth-First Search).md
            ./Algorithm/Algorithm Content/Graph Theory/BFS(Breadth-First Search).md
            ./Algorithm/Algorithm Content/Tree/Union Find.md
            Image Files
            ./Algorithm/Reference/Tree Reference/MST Ref/MST Graph.svg
            
            Example Answer 2
            Main file
            ./Algorithm/Algorithm Content/Graph Theory/BFS(Breadth-First Search).md
            Related Files
            NOTHING
            Image Files
            ./Algorithm/Reference/Graph Theory Reference/BASE TREE.svg
            ./Algorithm/Reference/Graph Theory Reference/BFS Ref/BFS Queue.svg

            Example Answer 3
            Main file
            ./Algorithm/a.md
            Related Files
            ./Algorithm/b.md
            ./Algorithm/c.md
            ./Algorithm/d.md
            ./Algorithm/e.md
            Image Files
            NOTHING            
            """,
            agent=agent,
            context=context,
            output_file="associateFilePath.md",
        )


    def research(self, agent, context):
        return Task(
            description="Based on the pathSearch, Gather and analyze the most relevant markdown files for troubleshooting {question}. Provide a summary of markdown files and what is an important point in files. If there is a symbol '[[file_name]]' in the markdown file, you should also look into the markdown file with the file_name.",
            expected_output="Your final answer MUST be a detailed summary of the markdown files in Korean. Include Path to all imported files, why the file was imported, what is an important point and If there is an example for {question}, include it. If you can't find the file, JUST Answer that you don't know",
            agent=agent,
            context=context,
            output_file="questionAnswer.md",
        )

In [7]:
from crewai import Crew

agent = Agents()
tasks = Tasks()

markdownPathSearcher = agent.markdownPathSearcher()
imgPathSearcher = agent.imgPathSearcher()
fileSelector = agent.fileSelector()
researcher = agent.researcher()

markdownPathSearcher_task = tasks.markdownPathSearch(markdownPathSearcher)
imgPathSearcher_task = tasks.imgPathSearch(imgPathSearcher)
fileSelector_task = tasks.fileSelct(
    fileSelector, [markdownPathSearcher_task, imgPathSearcher_task]
)
researcher_task = tasks.research(researcher, [markdownPathSearcher_task])

crew = Crew(
    agents=[
        markdownPathSearcher,
        imgPathSearcher,
        fileSelector,
    ],
    tasks=[
        markdownPathSearcher_task,
        imgPathSearcher_task,
        fileSelector_task,
    ],
    verbose=True,
)

result = crew.kickoff(
    dict(
        file_path=".\Algorithm\Algorithm Content",
        img_path=".\Algorithm\Reference",
        question="Fenwick 내용을 출력해줘",
    )
)

ValidationError: 1 validation error for Crew
verbose
  Input should be a valid boolean, unable to interpret input [type=bool_parsing, input_value=2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/bool_parsing